In [1]:
 import numpy as np
 from scipy.integrate import solve_ivp
 from numba import njit
 import jitr

In [2]:
 @njit
 def interaction(r, *args):
     (V0, W0, R0, a0) = args
     return jitr.woods_saxon_potential(r, (V0, W0, R0, a0))


In [17]:
sys = jitr.ProjectileTargetSystem(
    np.array([939.0]),
    np.array([5 * (2 * np.pi)]),
    l=np.array([0]),
    Ztarget=40,
    Zproj=0,
    nchannels=1,
)

Ecom = 14.1


# Lagrange-Mesh solver, don't set the energy
solver = jitr.LagrangeRMatrixSolver(40, 1, sys, ecom=None)

# use same interaction for all channels
interaction_matrix = jitr.InteractionMatrix(1)
interaction_matrix.set_local_interaction(interaction, 0, 0)

# Woods-Saxon potential parameters
V0 = 60  # real potential strength
W0 = 20  # imag potential strength
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness

params = (V0, W0, R0, a0)

In [4]:
ch = np.array(sys.build_channels(Ecom))
solver.set_energy(Ecom)

In [5]:
F = solver_lm.kernel.single_channel_free_matrix(ch[0])

In [6]:
A = solver_lm.kernel.single_channel_bloch_se_matrix(0,0,interaction, None,True,ch[0], params)

In [7]:
V = A - F # potential matrix elements 

In [8]:
R_lm, S_lm, x, uext_prime_boundary = solver_lm.solve(
    interaction_matrix, ch, args=params, ecom=Ecom
)

In [9]:
# Runge-Kutta
from jitr.utils import schrodinger_eqn_ivp_order1
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, ch[0], interaction_matrix.local_matrix[0, 0], params
    ),
    ch[0].domain,
    ch[0].initial_conditions(),
    dense_output=True,
    atol=1.0e-9,
    rtol=1.0e-9,
).sol

In [11]:
a = ch[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
print(R_rk)
S_rk = jitr.smatrix(R_rk, a, ch[0].l, ch[0].eta)
print(S_rk)

(-0.0021309110080514246+0.03561169340604122j)
(-0.05699944100839482-0.029794934193171304j)


In [12]:
R_lm

array([[0.0900102+3.23879482e-11j]])

In [13]:
import lagrange_rmatrix

In [14]:
sysLM = lagrange_rmatrix.ProjectileTargetSystem(
    incident_energy=Ecom,
    reduced_mass=sys.reduced_mass[0],
    channel_radius=sys.channel_radii[0],
    Ztarget=sys.Ztarget,
    Zproj=sys.Zproj,
)

In [15]:
sysLM

ProjectileTargetSystem(incident_energy=14.1, reduced_mass=939.0, channel_radius=31.41592653589793, Ztarget=40.0, Zproj=0.0, num_channels=1, level_energies=[])

In [16]:
chlm = lagrange_rmatrix.RadialSEChannel(0, sysLM, lambda r : interaction(r, params))